# Сравнение средних

*При анализе А/Б-тестов в мобильных приложениях и веб-сервисах часто возникает необходимость сравнить средние значения тех или иных метрик. Показано, как это можно сделать в рамках байесовского подхода.*

**Содержание:**
* [Введение](#Введение)  
* [Центральная предельная теорема](#Центральная-предельная-теорема)
* [Оценка среднего значения распределения](#Оценка-среднего-значения-распределения)
* [Сравнение средних двух распределений](#Сравнение-средних-двух-распределений)
* [Оценка длительности эксперимента до достижения заданного уровня уверенности](#Оценка-длительности-эксперимента-до-достижения-заданного-уровня-уверенности)
* [Приложение: сопряженное априорное распределение к нормальному распределению](#Приложение:-сопряженное-априорное-распределение-к-нормальному-распределению)
* [Приложение: оценка параметров нормального распределения по сэмплу](#Приложение:-оценка-параметров-нормального-распределения-по-сэмплу)
* [Заключение](#Заключение)  

# Введение

В веб-сервисы и мобильные приложения с течением времени вносятся изменения. Оценивать эффект от изменений можно сравнением метрик. Сравнение лучше проводить в рамках А/Б-тестов, чтобы минимизировать различие во внешних факторах.  

Наибольший интерес представляют метрики, заложенные в план развития продукта. Сравнение конверсий было рассмотрено в предыдущей части. Могут быть интересны денежные метрики и метрики вовлеченности - выручка на пользователя, время в приложении, длительность просмотра видео и т.д.  

*todo: картинка*

Байесовский подход [[SGBS](https://www.amazon.co.uk/Students-Guide-Bayesian-Statistics/dp/1473916364), [SR](https://www.amazon.co.uk/Statistical-Rethinking-Bayesian-Examples-Chapman/dp/036713991X/ref=sr_1_1)] требует построения моделей распределений сравниваемых величин. Выбор модели - неочевидный вопрос. Есть отдельные модели, получившие распространение - например, Buy Till You Die [[BTYD](https://en.wikipedia.org/wiki/Buy_Till_you_Die)] для количества покупок клиента за все время использования сервиса. Но универсальных моделей нет. Это создает сложности.

При этом для многих величин не всегда нужно знать все распределение. Можно ограничиться сравнением средних:
средней выручкой на пользователя, средней длительностью просмотра и т.д. Для средних значений часто применима центральная предельная теорема. Можно приближенно считать, что средние значения в выборках из распределения будут распределены нормально. Причем это не зависит от формы исходного распределения. Центральная предельная теорема позволяет при сравнении средних использовать нормальное распределение в качестве функции правдоподобия.

В ч.1 [[BayesAB1](https://nbviewer.org/github/noooway/Bayesian_ab_testing/blob/main/1-%D0%9E%D0%B1%D1%89%D0%B8%D0%B5%20%D0%B8%D0%B4%D0%B5%D0%B8.ipynb)] обсуждался байесовский подход к оценке А/Б-тестов. На примере конверсий было показано, как в рамках этого подхода ответить на вопросы
- Какой вариант лучше и насколько?
- Каковы оценки целевой метрики в каждом варианте?
- Насколько уверены в оценке?
- Сколько должен продолжаться эксперимент?

В этой части рассмотрено, как ответить на эти же вопросы применительно к сравнению средних. Структура следующая:
- вначале обсуждается центральная предельная теорема
- далее проводится оценка среднего значения по сэмплу данных из произвольного распределения с известными параметрами
- проводится сравнение средних для распределений с разными параметрами 
- предлагается способ оценки длительности эксперимента для сравнения средних значений

В приложениях приводятся детали расчета параметров сопряженного априорного распределения, расчет маржинальных распределений, пример оценки параметров нормального распределения по семплу.

# Центральная предельная теорема

Есть несколько центральных предельных теорем [[CLT](https://en.wikipedia.org/wiki/Central_limit_theorem)].
Одна из возможных формулировок следующая. Пусть есть последовательность независимых одинаково распределенных случайных величин $X_1, X_2, \dots, X_n, \dots$ с конечными математическим ожиданием $\mu$ и дисперсией $\sigma^2$. Пусть $\bar{X}_n = \frac{1}{n} \sum_{i=1}^{n} X_i$ их выборочное среднее. Тогда при $n$, стремящемся к бесконечности, распределение центрированных и масштабированных выборочных средних сходится к нормальному распределению [[NormDist](https://en.wikipedia.org/wiki/Normal_distribution), [SciPyNorm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html?highlight=norm)] со средним значением 0 и дисперсией 1

$$
\frac{\bar{X}_n - \mu}{(\sigma / \sqrt{n})} \to N(0, 1), \quad n \to \infty,
\\
N(x ; \mu, \sigma^2) = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{- \frac{(x-\mu)^2}{2 \sigma^2} } .
$$

Сходимость понимается как сходимость по распределению [[RandVarsConv](https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_distribution)].

Неформально этот результат можно применить следующим образом. Если взять произвольное распределение со средним значением $\mu$ и диспресий $\sigma^2$, начать выбирать из него сэмплы длины $n$ и считать среднее в каждом сэмпле, то средние значения сэмплов будут распределены приблизительно нормально $N(\mu, \sigma^2/n)$.

<center>
<img src="central_limit_theorem.png" alt="central_limit_theorem" width="600"/>
</center>

Центральная предельная теорема говорит о сходимости к нормальному распределению центрированных и масштабированных выборочных средних $\bar{X}_n$ при стремлении $n$ к бесконечности. Для фиксированного конечного числа $n$ нормальное распределение не гарантируется. При этом есть теоремы, дающие оценку отличия распределения суммы конечного количества случайных величин от нормального - см. [[BerryEsseenTheorem](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem)]. Отличие зависит как от количества слагаемых, так и от параметров распределения. 


*todo: на практике бывает достаточно нескольких десятков точек в каждом сэмпле.*  

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(7)

from collections import namedtuple

import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#todo: update scipy; make venv

In [ ]:
a = 1
sample_len = 100
n_samples = 1000

exact_dist = stats.gamma(a=a)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)

x = np.linspace(0, 10, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', name='Exact Dist'))
fig.add_trace(go.Histogram(x=samp[0], histnorm='probability density', name='Sample', nbinsx=20))
fig.update_layout(title=f'Original Distribution and a Sample of Length {sample_len}',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Means of Samples', nbinsx=50))
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=clt_mu, scale=clt_stdev), 
                         mode='lines', name='CLT Means Distrib'))
fig.update_layout(title='Sample Means Distribution',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

В приведенной формулировке центральная предельная теорема требует существования конечных среднего и дисперсии у исходного распределения. Примерами распределений, для которых эти свойства могут не выполнятся, являются распределение Парето [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution), [SciPyPareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html)] и близкое к нему распределение Ломакса [[LomaxDist](https://en.wikipedia.org/wiki/Lomax_distribution), [SciPyLomax](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lomax.html)]. Плотность вероятности последнего имеет вид

$$
P(x; c) = \frac{c}{(1 + x )^{c + 1}}, \quad x \ge 0, c > 0.
$$

При значениях параметра $c$ меньше или равном 2 дисперсия распределения Ломакса не является конечной.  

На графиках ниже приведена гистограмма средних в сэмплах и нормальное распределение с параметрами, равными среднему и дисперсии средних сэмплов.

In [ ]:
c = 1.7
sample_len = 500
n_samples = 1000

exact_dist = stats.lomax(c=c)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)
means_stdev = means.std()

x = np.linspace(0, 30, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', name='Exact Dist'))
fig.add_trace(go.Histogram(x=samp[0], histnorm='probability density', name='Sample'))
fig.update_layout(title=f'Original Distribution and a Sample of Length {sample_len}',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Means of Samples', nbinsx=120))
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=clt_mu, scale=means_stdev), 
                         mode='lines', name='CLT-like Normal'))
fig.update_layout(title='Sample Means Distribution',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

Видно, что распределение средних сильнее отличается от нормального, чем в предыдущем примере - оно скошено в сторону больших значений. Это объясняется тем, что плотность вероятности в распределениях Паретто и Ломакса "медленно" убывает с ростом $x$ ("тяжелый хвост"). Если в сэмпл попадает значение из хвоста, среднее сэмпла смещается. В итоге распределение средних перестает быть нормальным и становится скошенным в сторону больших значений.  

*todo: Есть обобщенная центральная предельная теорема [[GenCLT](https://en.wikipedia.org/wiki/Stable_distribution#A_generalized_central_limit_theorem)], которая говорит 
про предельное распределение средних в этом случае*.  

# Оценка среднего значения распределения

Пусть задано распределение и есть сэмпл из него. Нужно оценить среднее значение распределения.  

In [ ]:
a = 1
exact_dist = stats.gamma(a=a)

sample_size = 1000
samp = exact_dist.rvs(size=sample_size)
exact_mean = exact_dist.mean()
sample_mean = samp.mean()

x = np.linspace(0, 10, 2000)
ymax = np.max(exact_dist.pdf(x))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', name='Exact'))
fig.add_trace(go.Histogram(x=samp, histnorm='probability density', name='Sample', nbinsx=50))
fig.add_trace(go.Scatter(x=[exact_mean, exact_mean], y=[0, ymax], 
                         mode='lines', line_color='black', 
                         name='Exact Mean'))
fig.add_trace(go.Scatter(x=[sample_mean, sample_mean], y=[0, ymax], 
                         mode='lines', line_dash='dash', line_color='black', 
                         name='Sample Mean'))
fig.update_layout(title='Dist',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

Точечную оценку среднего распределения дает среднее значение в сэмпле. Но кроме точечной оценки интересно распределение возможных значений среднего. Для этого можно использовать центральную предельную теорему.

По центральной предельной теореме можно ожидать, что средние значения сэмплов будут распределены нормально. Поэтому можно выбрать нормальное распределение в качестве функции правдоподобия и с помощью соотношения Байеса построить распределение вероятности возможных параметров.  

Можно посчитать среднее по всем точкам сэмпла после чего оценивать распределения параметров модели для получения этого значения. Проблема в том, что по одной точке нельзя понять, выполняется ли центральная предельная теорема и можно ли применять нормальную функцию правдоподобия.

<center>
<img src="fit_overall_vs_subsample_means.png" alt="fit_overall_vs_subsample_means" width=550px/>
</center>

Более устойчивым способом кажется разбить исходные данные на части по несколько точек и в каждой части посчитать среднее. После чего оценить распределение параметров модели при приближении распределения средних нормальным распределением. В таком подходе появляется возможность валидировать модель.

Полезно построить распределение средних в частях и визуально сравнить его с нормальным. Вид гистограммы средних будет меняться в зависимости от выбранного количества точек $n_{split}$. Ниже выбрано фиксированное значение $n_{split}$ = 25.  

*todo: Можно попробовать оценить $n_{split}$ из неравенства Берри-Эссеена [[BerryEsseenTheorem](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem)]* .    
*Или подбирать $n_{split}$ как гиперпараметр и выбирать значение по итогам кросс-валидации.*  

Т.к. исходное распределение неизвестно, для параметров нормального распределения средних можно использовать среднее и дисперсию из выборки. 

In [ ]:
def reshape_and_compute_means(sample, n_split):
    n_means = len(sample) // n_split
    samp_reshaped = np.reshape(sample[0 : n_means * n_split], (n_means, n_split))
    means = np.array([x.mean() for x in samp_reshaped])
    return means

def exact_clt_dist(exact_dist, n_split):
    clt_mu = exact_dist.mean()
    clt_stdev = exact_dist.std() / np.sqrt(n_split)
    return stats.norm(loc=clt_mu, scale=clt_stdev)

def sample_clt_dist(means):
    clt_mu = means.mean()
    clt_std = means.std()
    return stats.norm(loc=clt_mu, scale=clt_std)

In [ ]:
n_split = 25
means = reshape_and_compute_means(samp, n_split)

x = np.linspace(0, 10, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Means of Samples'))
fig.add_trace(go.Scatter(x=x, y=sample_clt_dist(means).pdf(x), mode='lines', name='CLT-like Normal'))
fig.update_layout(title='Means of Samples and CLT-like Normal',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

*todo: Хочется иметь численный критерий, что можно использовать нормальное распределение.*  
*Либо критерий из Берри-Эссеена, либо кросс-валидация + нужно решить, что делать с ее результатами.*  
*Посмотреть, есть ли что-то в https://en.wikipedia.org/wiki/Normality_test .*     

В байесовском подходе для оценки плотности вероятности параметров модели используется соотношение

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)}.
$$

Для построения модели распределения средних байесовским методом функцию правдоподобия можно задать в виде нормального распределения:

$$
P(data | model) = N(x; \mu, \frac{\sigma_{0}^2}{\lambda} ) ,  
\\
N(x ; \mu, \frac{\sigma_{0}^2}{\lambda}) = \frac{\lambda^{1/2}}{\sqrt{2 \pi \sigma_{0}^2}} e^{- \frac{\lambda}{2 \sigma_{0}^2} (x-\mu)^2} .
$$

Можно показать (см. [приложение](#Приложение:-сопряженное-априорное-распределение-к-нормальному-распределению)), что для такой функции правдоподобия произведение нормального распределения для $\mu$ и гамма-распределения [[GammaDist](https://en.wikipedia.org/wiki/Gamma_distribution), [SciPyStatsGamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html)] для $\lambda$ будет сопряженным априорным распределением:
$$
P(model | data) = N(\mu; \mu_i, \frac{\sigma_0^2}{k_i \lambda} ) Gamma(\lambda; a_i, b_i) ,  
\\
Gamma(\lambda; a,b) = \frac{b^a}{\Gamma(a)} \lambda^{a-1} e^{-b \lambda} .
$$

Параметр $\mu$ характеризует центр распределения средних. Априорное распределение этого параметра для сопряженности выбирается нормальным, центр $\mu_i$ вычисляется по фактическим данным. Безразмерный масштабирующий множитель $\lambda$ входит в дисперсию средних $(\sigma_0^2 / \lambda)$ и дисперсию $\mu$ $(\sigma_0^2 / k_i \lambda )$. Параметры $a_i, b_i$ вычисляются по экспериментальным данным и определяют точное распределение этого коэффициента. Постоянная величина $\sigma_0$ задается по историческим данным. Можно ожидать, что по мере учета экспериментальных данных распределение $\mu$ будет локализовываться вокруг реального среднего, а отношение $\sigma_0^2 / \lambda$ - вокруг точной дисперсии $\sigma^2$.

<center>
<img src="conjugate_to_normal_lkh_parameters.png" alt="conjugate_to_normal_lkh_parameters" width=500px/>
</center>

Обновление параметров сопряженного распределения с каждой новой точкой $x_i$ выполняется
по соотношениям:

$$
k_{i+1} = k_i + 1 ,  
\\
\mu_{i+1} = \frac{x_i + k_i \mu_i}{k_i + 1} ,  
\\
a_{i+1} = a_i + 1/2 , 
\\
b_{i+1} = b_i + \frac{k_i}{k_i + 1} \frac{(x_i - \mu_i)^2}{2 \sigma_0^2} . 
$$

Исходные значения параметров $\mu_0$ и $\sigma_0$ можно задать произвольно. Но если они будут далеки
от реальных, то модель может оказаться неточной. Поэтому удобнее задавать их либо
по историческим данным, либо на основе части данных из сэмпла. Для $\mu_0$ можно
выбрать среднее $\bar{x}_1$ в первом наборе из $n_{split}$ точек, для $\sigma_0^2$ - дисперсию в этой
части сэмпла $\sigma_{\bar{x}_1}^2$, отнесенную к количеству точек $n_{split}$.
Для начальных значений $a_0$ и $b_0$ можно выбрать 2 и 1, поскольку при таких значениях
параметров мода $p(\lambda) = Gamma(\lambda, a=2, b=1)$ будет равна 1. Значение
$k_0$ можно выбрать $1/25$, чтобы вначале обеспечить широкую область для $P(\mu) = N(\mu_0, (5 \sigma_0)^2)$:

$$
\mu_0 = \bar{x}_1, \sigma_0 = \sigma_{\bar{x}_1} / \sqrt{n_{split}} ,
\\ 
k_0 = 1/25 ,
\\
a_0 = 2, b_0 = 1 .
$$

In [ ]:
ConjugateNormalParams = namedtuple('ConjugateNormalParams', 'mu sigma k a b')

def initial_parameters(mu, sigma, k=1/25, a=2, b=1):
    return ConjugateNormalParams(mu=mu, sigma=sigma, k=k, a=a, b=b)

def update_conj_parameters(x, conj_norm_pars):
    mu_p = (x + conj_norm_pars.k * conj_norm_pars.mu) / (conj_norm_pars.k + 1)
    sigma_p = conj_norm_pars.sigma
    k_p = conj_norm_pars.k + 1
    a_p = conj_norm_pars.a + 1/2
    b_p = conj_norm_pars.b + conj_norm_pars.k / (conj_norm_pars.k + 1) * (x - conj_norm_pars.mu)**2 / (2 * conj_norm_pars.sigma**2)
    return ConjugateNormalParams(mu=mu_p, sigma=sigma_p, k=k_p, a=a_p, b=b_p)

def compute_posterior_parameters(sample, n_split):
    means = reshape_and_compute_means(sample, n_split)
    mean_1 = means[0]
    sigma_1 = sample[0:n_split].std() / np.sqrt(n_split)
    pars = []
    pars.append(initial_parameters(mu=mean_1, sigma=sigma_1)) 
    for x in means[1:]:
        new_pars = update_conj_parameters(x, pars[-1])
        pars.append(new_pars)
    return pars

Для анализа параметров и сравнения групп вместо двумерного распределения параметров $P(\mu, \lambda | data)$ удобнее сравнивать одномерные маржинальные распределения по соответствующим параметрам $P(\lambda | data), P(\mu | data)$.

Маржинальное апостериорное распределение $P(\lambda | data)$ по конструкции задается гамма-распределением:

$$
P(\lambda | data) = Gamma(\lambda; a_i, b_i).
$$

Для интерпретации удобнее величина $\sigma = \sigma_0 / \lambda^{1/2}$, распределение которой можно получить с помощью замены переменных [[ProbChangeVars](https://en.wikipedia.org/wiki/Probability_density_function#Function_of_random_variables_and_change_of_variables_in_the_probability_density_function)]:

$$
P(\sigma | data) = Gamma\left( \frac{\sigma_0^2}{\sigma^2 }; a_i, b_i \right) \frac{2 \sigma_0^2}{\sigma^3}, 
\quad \sigma > 0 .
$$

Можно показать (см. [приложение](#Приложение:-сопряженное-априорное-распределение-к-нормальному-распределению)), что маржинальное апостериорное распределение $P(\mu | data)$ задается обобщенным $t$-распределением [[GenStudentT](https://en.wikipedia.org/wiki/Student's_t-distribution#Generalized_Student's_t-distribution), [SciPyT](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html)]:

$$
P(\mu | data) = t(\mu | \nu = 2a_i, \mu_t = \mu_i, \sigma_t^2 = \frac{\sigma_0^2}{k_i} \frac{b_i}{a_i} ).
$$

Для апостериорного распределения нужно вначале сгенерировать параметры модели, потом подставить их в функцию правдоподобия. Т.е. нужно произведение соответствующих распределений:

$$
P(x) = N(x; \mu, \frac{\sigma_0^2}{\lambda} ) N(\mu; \mu_i, \frac{\sigma_0^2}{k_i \lambda} ) Gamma(\lambda; a_i, b_i) .
$$

In [ ]:
def mu_marginal_distrib(conj_norm_pars):
    df = 2 * conj_norm_pars.a
    loc = conj_norm_pars.mu
    scale = np.sqrt(conj_norm_pars.sigma**2 / conj_norm_pars.k * conj_norm_pars.b / conj_norm_pars.a)
    return stats.t(df=df, loc=loc, scale=scale)

def lambda_marginal_distrib(conj_norm_pars):
    return stats.gamma(a=conj_norm_pars.a, scale=1/conj_norm_pars.b)

def sigma_marginal_distrib_pdf(x, conj_norm_pars):
    dl_ds = 2 * conj_norm_pars.sigma**2 / np.abs(x**3)
    return lambda_marginal_distrib(conj_norm_pars).pdf(conj_norm_pars.sigma**2 / x**2) * dl_ds
    
def post_means_rvs(conj_norm_pars, size):
    lmd = lambda_marginal_distrib(conj_norm_pars).rvs(size=size)
    post_mu = stats.norm.rvs(loc=conj_norm_pars.mu, scale=conj_norm_pars.sigma / np.sqrt(lmd * conj_norm_pars.k))
    post_means = stats.norm.rvs(loc=post_mu, scale=conj_norm_pars.sigma / np.sqrt(lmd))
    return post_means

Расчет оценок параметров с учетом экспериментальных данных:

In [ ]:
pars = compute_posterior_parameters(samp, n_split)
print(f'Initial pars: {pars[0]}')
print(f'Final pars: {pars[-1]}')

Распределения параметров $\mu$ и $\sigma$ приведены ниже:

In [ ]:
#mu
mu_dist = mu_marginal_distrib(pars[-1])
x = np.linspace(0, 10, 2000)
yplot = mu_dist.pdf(x)
ymax = max(yplot)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=mu_dist.pdf(x), mode='lines', name=f'Mu Estimate Distrib'))
fig.add_trace(go.Scatter(x=[means.mean(), means.mean()], y=[0, ymax],
                         mode='lines', line_color='black', line_dash='dash', 
                         name='Sample Mean'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, ymax], 
                         mode='lines', line_color='black',
                         name='Exact Mean'))
fig.update_layout(title='Dist',
                  xaxis_title='mu',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

# sigma
fig = go.Figure()
x = np.linspace(0.001, 10, 2000)
x_plot = x
y_plot = sigma_marginal_distrib_pdf(x, pars[-1])
y_max = max(y_plot)
fig.add_trace(go.Scatter(x=x, 
                         y=y_plot, 
                         mode='lines', line_dash='dash', name=f'Sigma Estimate Distrib'))
fig.add_trace(go.Scatter(x=[means.std(), means.std()], 
                         y=[0, y_max],
                         mode='lines', line_color='black', line_dash='dash', 
                         name='Sample Stdev'))
fig.add_trace(go.Scatter(x=[exact_clt_dist(exact_dist, n_split).std(), exact_clt_dist(exact_dist, n_split).std()], 
                         y=[0, y_max], 
                         mode='lines', line_color='black',
                         name='Exact Stdev'))
fig.update_layout(title='Dist',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.update_layout(xaxis_range=[0, 1])
fig.show()

Видно, что моды распределений близки к значениям среднего и дисперсии в сэмпле. Также точные значения имеют "большое" значение плотности вероятности.

Распределение сгенерированных апостериорных средних:

In [ ]:
#post means
post_means = post_means_rvs(pars[-1], size=50000)
x = np.linspace(0, 10, 2000)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.add_vline(means.mean(), line_dash='dash', name='Sample Mean')
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Means of Samples',
                           opacity=0.7, marker_color='green'))
#fig.add_trace(go.Scatter(x=x, y=exact_clt_dist(exact_dist, n_split).pdf(x), mode='lines', name='CLT Distribution'))
fig.add_trace(go.Scatter(x=x, y=sample_clt_dist(means).pdf(x), mode='lines', name='CLT-like Distribution'))
fig.add_trace(go.Scatter(x=x, y=mu_dist.pdf(x), mode='lines', name=f'Mu Estimate Distrib'))    
   
fig.add_trace(go.Histogram(x=post_means, histnorm='probability density', name='Posterior Means', 
                           opacity=0.7, nbinsx=100))
fig.update_layout(title='Dist',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.update_layout(xaxis_range=[0, 5])
fig.show()

Распределение сгенерированных апостериорных средних обычно шире, чем распределение центральной предельной теоремы. Это объясняется дисперсией в параметрах $\mu$ и $\lambda$ (в центральной предельной теореме $\mu$ и $\sigma$ фиксированные числа). 

Для наглядности удобно нарисовать распределение параметра $\mu_i$ и распределение апостериорных средних на одном графике. Распределение средних шире, чем распределение параметра $\mu$. Это связано с учетом дисперсии $\sigma_0^2/\lambda$. 

# Сравнение средних двух распределений

Пусть есть два распределения одинаковой формы, но с разными средними значениями.
Для примера

$$
x \sim Gamma(a, b),
\quad
\bar{x} = \frac{a}{b} .
$$

В одном случае параметры $a_A = 1, b_A = 1$ и среднее $\bar{x}_A = 1$, в другом параметры $a_B = 1, b_B = 1.25$ и среднее $\bar{x}_B = 0.8$. При выбранных значениях параметров среднее в группе $B$ отличается на $20\%$ от группы $A$ (относительно $A$).

In [ ]:
exp_info = pd.DataFrame([
    {'group':'A', 'a':1, 'b':1, 'sample_size':10000},
    {'group':'B', 'a':1, 'b':1.25, 'sample_size':10000}])
exp_info.set_index('group', inplace=True)

exact_dist_a = stats.gamma(a=exp_info['a']['A'], scale=1/exp_info['b']['A'])
exact_dist_b = stats.gamma(a=exp_info['a']['B'], scale=1/exp_info['b']['B'])
exp_info['exact_mean'] = pd.Series({'A': exact_dist_a.mean(), 'B':exact_dist_b.mean()})
display(exp_info)

samp_a = exact_dist_a.rvs(size=exp_info['sample_size']['A'])
samp_b = exact_dist_b.rvs(size=exp_info['sample_size']['B'])

x = np.linspace(0, 10, 2000)
ymax = np.max([exact_dist_a.pdf(x), exact_dist_b.pdf(x)])

fig = go.Figure()
col = 'red'
fig.add_trace(go.Scatter(x=x, y=exact_dist_a.pdf(x), 
                         mode='lines', line_color=col,
                         name=f"Exact A: a={exp_info['a']['A']}, b={exp_info['b']['A']}"))
fig.add_trace(go.Histogram(x=samp_a, histnorm='probability density',
                           name='Sample A',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=[exact_dist_a.mean(), exact_dist_a.mean()], y=[0, ymax], 
                         mode='lines', line_color=col, line_dash='dash',
                         name='Exact Mean A'))
col = 'blue'
fig.add_trace(go.Scatter(x=x, y=exact_dist_b.pdf(x), 
                         mode='lines', line_color=col,
                         name=f"Exact B: a={exp_info['a']['B']}, b={exp_info['b']['B']}"))
fig.add_trace(go.Histogram(x=samp_b, histnorm='probability density',
                           name='Sample B',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=[exact_dist_b.mean(), exact_dist_b.mean()], y=[0, ymax], 
                         mode='lines', line_color=col, line_dash='dash',
                         name='Exact Mean B'))
fig.update_layout(title='Exact Distributions and Samples',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode='overlay')
fig.show()

Распределение средних на основе точек из сэмплов:

In [ ]:
n_split = 25

means_a = reshape_and_compute_means(samp_a, n_split)
means_b = reshape_and_compute_means(samp_b, n_split)

x = np.linspace(0, 10, 2000)
ymax = np.max([exact_dist_a.pdf(x), exact_dist_b.pdf(x), 
               sample_clt_dist(means_a).pdf(x), sample_clt_dist(means_b).pdf(x)])


fig = go.Figure()
col = 'red'
fig.add_trace(go.Histogram(x=means_a, histnorm='probability density',
                           name='A Sample Means',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=x, y=sample_clt_dist(means_a).pdf(x), 
                         mode='lines', line_color=col, opacity=0.5,
                         name=f"A Sample CLT"))
fig.add_trace(go.Scatter(x=x, y=exact_dist_a.pdf(x), 
                         mode='lines', line_color=col, line_dash='dash', opacity=0.3,
                         name=f"A Exact: a={exp_info['a']['A']}, b={exp_info['b']['A']}"))
col = 'blue'
fig.add_trace(go.Histogram(x=means_b, histnorm='probability density',
                           name='B Sample Means',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=x, y=sample_clt_dist(means_b).pdf(x), 
                         mode='lines', line_color=col, opacity=0.5,
                         name=f"B Sample CLT"))
fig.add_trace(go.Scatter(x=x, y=exact_dist_b.pdf(x), 
                         mode='lines', line_color=col, line_dash='dash', opacity=0.3,
                         name=f"B Exact: a={exp_info['a']['B']}, b={exp_info['b']['B']}"))
fig.update_layout(title='Sample Means',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  barmode='overlay',
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()
#
exp_info['exact_mean'] = pd.Series({'A': exact_dist_a.mean(), 'B': exact_dist_b.mean()})
exp_info['sample_mean'] = pd.Series({'A': samp_a.mean(), 'B': samp_b.mean()})
exp_info

Расчет параметров с учетом данных:

In [ ]:
pars_a = compute_posterior_parameters(samp_a, n_split)
pars_b = compute_posterior_parameters(samp_b, n_split)

print(pars_a[-1])
print(pars_b[-1])

Распределения параметров $\mu$, $\sigma$ и апостериорных средних:

In [ ]:
#mu
mu_dist_a = mu_marginal_distrib(pars_a[-1])
mu_dist_b = mu_marginal_distrib(pars_b[-1])
x_mu = np.linspace(0, 10, 2000)
ymax_mu = np.max([mu_dist_a.pdf(x_mu), mu_dist_b.pdf(x_mu)]) #todo: optimize

#sigma
x_sg = np.linspace(0.01, 10, 2000)
sigma_dist_a = sigma_marginal_distrib_pdf(x_sg, pars_a[-1])
sigma_dist_b = sigma_marginal_distrib_pdf(x_sg, pars_b[-1])
ymax_sg = np.max([sigma_dist_a, sigma_dist_b]) #todo: optimize

#posterior
post_means_a = post_means_rvs(pars_a[-1], size=30000)
post_means_b = post_means_rvs(pars_b[-1], size=30000)
x_post = np.linspace(0, 10, 2000)


#mu
fig = go.Figure()
col = 'red'
fig.add_trace(go.Scatter(x=x_mu, y=mu_dist_a.pdf(x), 
                         mode='lines', line_color=col, 
                         name=f'A Mu Dist'))
fig.add_trace(go.Scatter(x=[means_a.mean(), means_a.mean()], y=[0, ymax_mu],
                         mode='lines', line_color=col, line_dash='dash', 
                         name='A Sample Mean'))
col = 'blue'
fig.add_trace(go.Scatter(x=x_mu, y=mu_dist_b.pdf(x), 
                         mode='lines', line_color=col, 
                         name=f'B Mu Dist'))
fig.add_trace(go.Scatter(x=[means_b.mean(), means_b.mean()], y=[0, ymax_mu],
                         mode='lines', line_color=col, line_dash='dash', 
                         name='B Sample Mean'))
fig.update_layout(title='Mu Estimates',
                  xaxis_title='mu',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 2])
fig.show()


#sigma
fig = go.Figure()
col = 'red'
fig.add_trace(go.Scatter(x=x_sg, y=sigma_dist_a, 
                         mode='lines', line_color=col, 
                         name=f'A Sigma Dist'))
fig.add_trace(go.Scatter(x=[means_a.std(), means_a.std()], y=[0, ymax_sg],
                         mode='lines', line_color=col, line_dash='dash', 
                         name='A Sample Std'))
col = 'blue'
fig.add_trace(go.Scatter(x=x_sg, y=sigma_dist_b,
                         mode='lines', line_color=col, 
                         name=f'B Sigma Dist'))
fig.add_trace(go.Scatter(x=[means_b.std(), means_b.std()], y=[0, ymax_sg],
                         mode='lines', line_color=col, line_dash='dash', 
                         name='B Sample Std'))
fig.update_layout(title='Sigma Estimates',
                  xaxis_title='Sigma',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 2])
fig.show()


#posterior
fig = go.Figure()
col = 'red'
fig.add_trace(go.Scatter(x=x_post, y=exact_dist_a.pdf(x_post), 
                         mode='lines', line_dash='dash', line_color=col, opacity=0.3,
                         name='A Exact'))
fig.add_trace(go.Scatter(x=x_post, y=sample_clt_dist(means_a).pdf(x_post), 
                         mode='lines', line_color=col, opacity=0.5,
                         name=f"A Sample CLT"))
fig.add_trace(go.Histogram(x=post_means_a, histnorm='probability density',
                           opacity=0.3, marker_color=col,
                           name='A Posterior Means'))
col = 'blue'
fig.add_trace(go.Scatter(x=x_post, y=exact_dist_b.pdf(x_post), 
                         mode='lines', line_dash='dash', line_color=col, opacity=0.3,
                         name='B Exact'))
fig.add_trace(go.Scatter(x=x_post, y=sample_clt_dist(means_b).pdf(x_post), 
                         mode='lines', line_color=col, opacity=0.5,
                         name=f"B Sample CLT"))
fig.add_trace(go.Histogram(x=post_means_b, histnorm='probability density', 
                           opacity=0.3, marker_color=col,
                           name='B Posterior Means'))
fig.update_layout(title='Posterior Sample Means',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.update_layout(xaxis_range=[0, 5])
fig.show()

Распределения апостериорных средних существенно пересекаются, чего не происходит с распределениями $\mu$ и $\sigma$. 

Для выбора одной из групп нужно сравнить распределения. Для этого посмотреть отношение и также оценить вероятности того, что значение параметров в одной из групп больше другой.

При этом возникает вопрос, на какие распределения ориентироваться при выборе варианта - распределения $\mu$ или апостериорных средних? Что делать с $\sigma$? Можно ожидать, что по мере набора данных распределение $\mu_i$ будет сужаться с центром на реальном среднем. Распределение апострериорных средних характеризует средние значения в сэмпле из $n_{split}$ точек. Это не то же самое, что истинное среднее - у этой величины больше дисперсия (поэтому распределения апостериорных средних на графиках выше перекрываются). При сравнении групп нужно выбрать группу именно с большим средним, поэтому нужно ориентироваться на распределение $\mu$. Т.к. распределение $\mu$ проинтегрировано по $\lambda$, то оно не зависит от $\sigma = \sigma_0 / \lambda^{1/2}$. В распределении $\sigma$ полезно проверить, что стандартное отклонение сэмпла лежит вблизи моды этого распределения - если это не так, то модель, возможно, некорректна. В-остальном, $\sigma$ не используется для выбора групп. 

*todo: еще возникает вопрос с зависимостью оценки от $n_{split}$*.  

Для оценки вероятностей того, что значения в одной группе больше другой можно сгенерировать и сравнить сэмплы соответствующих распределений.

In [ ]:
def p_mu_b_ge_mu_a(pars_a, pars_b, mu_size=30000):
    mu_dist_a = mu_marginal_distrib(pars_a)
    mu_dist_b = mu_marginal_distrib(pars_b)
    return np.sum(mu_dist_b.rvs(size=mu_size) >= mu_dist_a.rvs(size=mu_size)) / mu_size

def p_sigma_b_ge_sigma_a(pars_a, pars_b, sg_size=30000):
    lambdas_a = lambda_marginal_distrib(pars_a).rvs(size=sg_size)
    lambdas_b = lambda_marginal_distrib(pars_b).rvs(size=sg_size)
    sigmas_a = pars_a.sigma / np.sqrt(lambdas_a)
    sigmas_b = pars_b.sigma / np.sqrt(lambdas_b)
    return np.sum(sigmas_b >= sigmas_a) / sg_size

def p_expected_b_ge_expected_a(pars_a, pars_b, post_size=30000):
    post_means_a = post_means_rvs(pars_a, size=post_size)
    post_means_b = post_means_rvs(pars_b, size=post_size)
    return np.sum(post_means_b >= post_means_a) / post_size

In [ ]:
print(f'P(mu_B > mu_A): {p_mu_b_ge_mu_a(pars_a[-1], pars_b[-1])}')
print(f'P(sigma_B > sigma_A): {p_sigma_b_ge_sigma_a(pars_a[-1], pars_b[-1])}')
print(f'P(E[B] > E[A]): {p_expected_b_ge_expected_a(pars_a[-1], pars_b[-1])}')

Также для сравнения может быть удобно построить отношение распределений.

In [ ]:
#mu rel
mu_dist_a = mu_marginal_distrib(pars_a[-1]).rvs(size=10000)
mu_dist_b = mu_marginal_distrib(pars_b[-1]).rvs(size=10000)
mu_rel = mu_dist_b / mu_dist_a

fig = go.Figure()
fig.add_trace(go.Histogram(x=mu_rel, histnorm='probability density',
                           name='Mu Relation'))
fig.add_vline(x=1, line_dash='dash')
fig.update_layout(title='Dist',
                  xaxis_title='mu_B / mu_A',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.update_layout(xaxis_range=[0, 2])
fig.show()

#post means rel
post_means_a = post_means_rvs(pars_a[-1], size=30000)
post_means_b = post_means_rvs(pars_b[-1], size=30000)
post_means_rel = post_means_b / post_means_a

fig = go.Figure()
fig.add_trace(go.Histogram(x=post_means_rel, histnorm='probability density',
                           name='Posterior Means Relation', nbinsx=100))
fig.add_vline(x=1, line_dash='dash')
fig.update_layout(title='Dist',
                  xaxis_title='Posterior B/Posterior A',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.update_layout(xaxis_range=[0, 5])
fig.show()


print(f"E[mu_B/mu_A] = {mu_rel.mean()}")
print(f"Posterior E[E[B]/E[A]] = {post_means_rel.mean()}")

print(f"Exact E[B]/E[A] = {exp_info['exact_mean']['B'] / exp_info['exact_mean']['A']}")
print(f"Sample E[B]/E[A] = {exp_info['sample_mean']['B'] / exp_info['sample_mean']['A']}")

Сводная таблица со сравнением групп:

In [ ]:
exp_formatted = pd.DataFrame(index=exp_info.index)
exp_formatted['Sample Size'] = exp_info['sample_size'].apply(lambda x: f'{x}')
exp_formatted['E[mu]'] = pd.Series({'A': mu_dist_a.mean(), 'B': mu_dist_b.mean()}).apply(lambda x: f'{x:.2f}')
exp_formatted['E[mu] Relative to A'] = pd.Series({'A': 1, 'B': (mu_dist_b/mu_dist_a).mean()}).apply(lambda x: f'{x:.2f}')
tmp = p_mu_b_ge_mu_a(pars_a[-1], pars_b[-1])
exp_formatted['Prob(Highest E[mu]), %'] = pd.Series({'A': 1 - tmp, 'B': tmp}).apply(lambda x: f'{x*100:.1f}')
exp_formatted.T

# Оценка длительности эксперимента до достижения заданного уровня уверенности

Предположим, что по имеющимся данным оценки средних в группах близки.  
Сколько еще нужно продолжать эксперимент, пока уверенность в различии групп не достигнет заданного уровня?

In [ ]:
exp_info = pd.DataFrame([
    {'group':'A', 'a':1, 'b':1, 'sample_size': 300},
    {'group':'B', 'a':1, 'b':1.05, 'sample_size': 300}])
exp_info.set_index('group', inplace=True)

exact_dist_a = stats.gamma(a=exp_info['a']['A'], scale=1/exp_info['b']['A'])
exact_dist_b = stats.gamma(a=exp_info['a']['B'], scale=1/exp_info['b']['B'])
exp_info['exact_mean'] = pd.Series({'A': exact_dist_a.mean(), 'B':exact_dist_b.mean()})
display(exp_info)

samp_a = exact_dist_a.rvs(size=exp_info['sample_size']['A'])
samp_b = exact_dist_b.rvs(size=exp_info['sample_size']['B'])

pars_a = compute_posterior_parameters(samp_a, n_split)
pars_b = compute_posterior_parameters(samp_b, n_split)

#mu
mu_dist_a = mu_marginal_distrib(pars_a[-1])
mu_dist_b = mu_marginal_distrib(pars_b[-1])
x_mu = np.linspace(0, 10, 2000)
ymax_mu = np.max([mu_dist_a.pdf(x_mu), mu_dist_b.pdf(x_mu)]) #todo: optimize

fig = go.Figure()
col = 'red'
fig.add_trace(go.Scatter(x=x_mu, y=mu_dist_a.pdf(x), 
                         mode='lines', line_color=col, 
                         name=f'A Mu Dist'))
fig.add_trace(go.Scatter(x=[samp_a.mean(), samp_a.mean()], y=[0, ymax_mu],
                         mode='lines', line_color=col, line_dash='dash', 
                         name='A Sample Mean'))
col = 'blue'
fig.add_trace(go.Scatter(x=x_mu, y=mu_dist_b.pdf(x), 
                         mode='lines', line_color=col, 
                         name=f'B Mu Dist'))
fig.add_trace(go.Scatter(x=[samp_b.mean(), samp_b.mean()], y=[0, ymax_mu],
                         mode='lines', line_color=col, line_dash='dash', 
                         name='B Sample Mean'))
fig.update_layout(title='Mu Estimates',
                  xaxis_title='mu',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 2])
fig.show()

Для оценки дополнительного количества точек до достижения определенного уровня уверенности можно применить моделирование. По текущим данным есть оценки параметров $(a_i, b_i, k_i, \mu_i, \sigma_0)$ для распределений $P(\lambda) = Gamma(\lambda; a_i, b_i)$ и $P(\mu) = N(\mu; \mu_i, \sigma_0^2 / (k_i \lambda) ) Gamma(\lambda; a_i, b_i)$ для обеих групп. Можно сгенерировать сэмплы $\mu$ и $\lambda$ и предположить, что это реальные значения. С этими значениями сгенерировать дополнительные апостериорные средние $P(x) = N(x; \mu, \sigma_{0}^2/\lambda )$. Далее предположить, что именно такие данные получались бы при реальном продолжении эксперимента и использовать эти апостериорные средние для переоценки параметров распределений $P(\lambda)$ и $P(\mu)$. Нужно сделать несколько таких моделирований. Для каждого моделирования для каждой дополнительной точки посчитать $P(\mu_B \ge \mu_A | data)$ и определить минимальное количество дополнительных точек $N_{min}$, которое потребовалось для достижения заданного уровня уверенности $p$, т.е. $N_{min} : P(\mu_B \ge \mu_A | data+sim) \ge p$ или $P(\mu_B \ge \mu_A | data+sim ) \le 1 - p$. По гистограмме $N_{min}$ можно оценить дополнительное количество данных. Для получения оценки дополнительного количества реальных наблюдений количество дополнительных апостериорных средних нужно домножить на $n_{split}$ .

In [ ]:
def simulate(pars, sim_posteriors_max_len):
    lmd = stats.gamma.rvs(a=pars.a, scale=1/pars.b)
    mu = stats.norm.rvs(loc=pars.mu, scale=pars.sigma / np.sqrt(lmd * pars.k))
    posteriors = stats.norm.rvs(loc=mu, scale=pars.sigma / np.sqrt(lmd), size=sim_posteriors_max_len)
    sim_pars = []
    sim_pars.append(pars)
    for x in posteriors:
        sim_pars.append(update_conj_parameters(x, sim_pars[-1]))
    s = {'mu': mu, 'lmd': lmd, 'sim_posteriors_max_len': sim_posteriors_max_len,
         'sim_pars': sim_pars, 'post_means': posteriors}
    return s

def p_mub_gt_mua_sims(s_a, s_b, n_cmp=30000):
    p_mub_gt_mua = []
    for pars_a, pars_b in zip(s_a['sim_pars'], s_b['sim_pars']):
        mu_dist_a = mu_marginal_distrib(pars_a).rvs(size=n_cmp)
        mu_dist_b = mu_marginal_distrib(pars_b).rvs(size=n_cmp)
        p_mub_gt_mua.append(np.sum(mu_dist_b > mu_dist_a) / n_cmp)
    return np.array(p_mub_gt_mua)

def min_points_to_reach_certainty_level(probs_pb_ge_pa, n_point, required_pb_ge_pa):
    prob_gt_required = (probs_pb_ge_pa > required_pb_ge_pa) | (probs_pb_ge_pa < 1 - required_pb_ge_pa)
    reached = n_point[prob_gt_required]
    min_reached = np.min(reached) if prob_gt_required[-1] else np.max(n_point)
    return min_reached

def run_simulations(pars_start_a, pars_start_b, p_lvl=0.95, sim_max_len=50, n_sims=100):
    sims = []
    for i in range(n_sims):
        s_a = simulate(pars_start_a, sim_max_len)
        s_b = simulate(pars_start_b, sim_max_len)
        p_mub_gt_mua = p_mub_gt_mua_sims(s_a, s_b)
        min_reached = min_points_to_reach_certainty_level(p_mub_gt_mua, np.arange(len(p_mub_gt_mua)), p_lvl)
        sims.append({'i': i, 'A': s_a, 'B': s_b, 'p_mub_gt_mua': p_mub_gt_mua, 'min_reached': min_reached})
    return sims

Моделирование различных сценариев:

In [ ]:
p_lvl = 0.95
sims = run_simulations(pars_start_a=pars_a[-1], pars_start_b=pars_b[-1], p_lvl=p_lvl)
#print(sims[99])

Динамика $P(\mu_B \ge \mu_A | data)$ по фактическим данным и смоделированные эксперименты:

In [ ]:
pb_gt_pa = [p_mu_b_ge_mu_a(a, b) for a, b in zip(pars_a, pars_b)]
fact_len = len(pars_a)
n_points = [i * n_split for i in range(1, fact_len + 1)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=n_points, y=pb_gt_pa,
                         name='Fact',
                         line_color='blue'))
for s in sims:
    fig.add_trace(go.Scatter(x=np.arange(fact_len, fact_len + len(s['p_mub_gt_mua']) + 1) * n_split, 
                             y=s['p_mub_gt_mua'],
                             name=f"Sim {s['i']}",
                             line_color='red',
                             opacity=0.1))
fig.add_hline(y=0.05, line_dash='dash')
fig.add_hline(y=0.95, line_dash='dash')
fig.update_layout(title='Simulations P(mu_b >= mu_a)',
                  xaxis_title='N Points',
                  yaxis_title='P(mu_b >= mu_a)',
                  hovermode="x",
                  height=550,
                  barmode="overlay",
                  showlegend=False)
fig.update_layout(xaxis_range=[0, (fact_len + len(sims[0]['p_mub_gt_mua'])) * n_split], yaxis_range=[0,1])
fig.show()

Гистограмма минимального количества дополнительных точек, нужных для достижения заданного уровня уверенности:

In [ ]:
n_reached_hist = [s['min_reached'] * n_split for s in sims]
n_reached_freqs = pd.Series(n_reached_hist).value_counts(normalize=True).rename('freq').to_frame()
x_med = np.median(n_reached_hist)

fig = go.Figure()
fig.add_trace(go.Bar(x=n_reached_freqs.index,
                     y=n_reached_freqs['freq'],
                     width=[n_split] * len(n_reached_freqs),
                     marker_color='red',
                     opacity=0.6,
                     name='Simulations Reached Certainty'))
fig.add_trace(go.Scatter(x=[x_med, x_med], y=[0, np.max(n_reached_freqs['freq'])], 
                         line_color='black',
                         line_dash='dash',
                         mode='lines',
                         hovertemplate=f"Median: {x_med}",
                         name='Median'))              
fig.update_layout(title=f'Additional Points to Reach {p_lvl*100:.0f}% Certainty')
fig.update_layout(xaxis_title='Additional Points',
                  yaxis_title='Part from Total Simulations',
                  showlegend=False)
fig.update_xaxes(range=[0, (len(sims[0]['p_mub_gt_mua']) + 1) * n_split])
fig.update_layout(yaxis_rangemode='tozero')
fig.show()

Видно, что в 58% моделирований текущего максимального количества точек в моделировании не хватает для достижения уверенности, а в 42% моделирований уверенность доходит до заданного значения при различном количестве дополнительных точек.

**todo: Продолжение эксперимента до снижения ценности новых данных**

**todo: Количество правильно угаданных вариантов**

# Приложение: сопряженное априорное распределение к нормальному распределению

В байесовском подходе для оценки плотности вероятности параметров модели используется соотношение

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)}.
$$

Пусть функция правдоподобия $P(data|model)$ задается нормальным распределением

$$
P(data | model) = N(x ; \mu, \frac{\sigma_0^2}{\lambda}) = \frac{\lambda^{1/2}}{\sqrt{2 \pi \sigma_0^2}} 
e^{- \frac{\lambda}{2 \sigma_0^2}(x - \mu)^2}.
$$

Можно показать [[ConjNormal](https://www.cs.ubc.ca/~murphyk/Papers/bayesGauss.pdf)], что произведение нормального распределения по $\mu$ и гамма-распределения по $\lambda$ будет сопряженным априорным распределением к функции правдоподобия выше

$$
P(model) = N(\mu ; \mu_i, \frac{\sigma_0^2}{k_i \lambda}) Gamma(\lambda; a_i, b_i),
$$
где
$$
Gamma(\lambda; a, b) = \frac{b^a}{\Gamma(a)} \lambda^{a-1} e^{-b \lambda}, \quad x>0, \quad a,b>0 .
$$

Для сопряженности нужно, чтобы $P(model \mid data)$ имела ту же зависимость от $\mu$ и $\lambda$, что и $P(model)$, но с другими значениями параметров $(a_{i}, b_{i}, \mu_{i}, k_{i})$. Новые параметры $(a_{i+1}, b_{i+1}, \mu_{i+1}, k_{i+1})$ должны выражаться через старые значения $(a_i, b_i, \mu_i, k_i)$ и фактические данные $x$. При подстановке:

\begin{split}
P(model \mid data) 
& = 
P(\mu, \lambda | x ) 
\\
& \propto  
N(x ; \mu, \frac{\sigma_0^2}{\lambda})
N(\mu ; \mu_i, \frac{\sigma_0^2}{k_i \lambda}) 
Gamma(\lambda; a_i, b_i)
\\
& \propto
\frac{\lambda^{1/2}}{\sqrt{2 \pi \sigma_0^2}} e^{- \frac{\lambda}{2 \sigma_0^2}(x - \mu)^2}
\frac{(k_i \lambda)^{1/2}}{\sqrt{2 \pi \sigma_0^2}} e^{- \frac{k_i \lambda}{2 \sigma_0^2}(\mu - \mu_i)^2}
\frac{b_i^{a_i}}{\Gamma(a_i)} e^{-b_i \lambda} {\lambda}^{a_i-1} .
\end{split}

Выражение в показателе экспонент можно записать в виде

$$
\begin{align}
\frac{\lambda}{2 \sigma_0^2}(x - \mu)^2 + 
\frac{k_i \lambda}{2 \sigma_0^2}(\mu - \mu_i)^2 +
\lambda b_i
& =
\frac{\lambda}{2 \sigma_0^2}
\left( \mu^2(k_i+1) - 2\mu(x + k_i \mu_i) \right) + 
\frac{\lambda}{2 \sigma_0^2}(x^2 + k_i \mu_i^2 + 2 b_i \sigma_0^2)
\\
& =
\frac{\lambda (k_i + 1)}{2 \sigma_0^2}
\left( \mu - \frac{x + k_i \mu_i}{k_i + 1} \right)^2
+
\frac{\lambda}{2 \sigma_0^2} \left( x^2 + k_i \mu_i^2 + 2 b_i \sigma_0^2 - \frac{(x + k_i \mu_i)^2}{k_i + 1} \right)
\\
& =
\frac{\lambda (k_i + 1)}{2 \sigma_0^2}
\left( \mu - \frac{x + k_i \mu_i}{k_i + 1} \right)^2
+
\frac{\lambda}{2 \sigma_0^2} \left( \frac{k_i}{k_i+1} (x - \mu_i)^2 + 2 b_i \sigma_0^2 \right) .
\end{align} 
$$

После подстановки факторы, зависящие только от $\mu$ и $\lambda$:

$$
P(\mu, \lambda | x ) 
\propto 
\lambda^{1/2} e^{- \frac{\lambda (k_i + 1)}{2 \sigma_0^2} \left( \mu - \frac{x + k_i \mu_i}{k_i+1} \right)^2}
{\lambda}^{a_i-1/2} 
e^{- \frac{\lambda}{2 \sigma_0^2} \left( \frac{k_i}{k_i + 1} (x - \mu_i)^2 + 2 b_i \sigma_0^2 \right) } .
$$


Апостериорное распределение имеет вид

$$
P(\mu, \lambda | x ) = N(\mu; \mu_{i+1}, \frac{\sigma_0^2}{k_{i+1} \lambda} ) Gamma(\lambda; a_{i+1}, b_{i+1}) .
$$

Новые значения параметров связаны со старыми соотношениями:
$$
k_{i+1} = k_i + 1 ,
\\
\mu_{i+1} = \frac{x + k_i \mu_i}{k_i + 1} , 
\\
a_{i+1} = a_i + 1/2 ,
\\
b_{i+1} = b_i + \frac{k_i}{k_i + 1} \frac{(x - \mu_i)^2}{2 \sigma_0^2} .
$$

У нормальной функции правдоподобия два параметра - среднее $\mu$ и дисперсия $\sigma_0^2/\lambda$. Априорное распределение $\mu$ также выбирается нормальным; центр этого распределения $\mu_i$ определяется по фактическим данным. В дисперсии $\sigma_0^2/\lambda$ параметр $\sigma_0$ постоянный и задается априорно. Безразмерный параметр $\lambda$ варьируется. Его распределение задается гамма-распределением. Этот параметр также регулирует дисперсию $\mu$ ( $\sigma_0^2 / k_i \lambda$ ). 

Можно ожидать, что с ростом количества экспериментальных данных распределение $\mu$ будет локализовываться вокруг реального среднего, а отношение $\sigma_0^2 / \lambda$ - вокруг точной дисперсии $\sigma^2$. Центр распределения $\mu$ - значение $\mu_{i+1}$ - строится как взвешенное среднее между новой точкой $x$ и предыдущим значением $\mu_i$. Дисперсия зависит от параметра $\lambda$, распределение которого в свою очередь зависит от $a_i$ и $b_i$. При добавлении новой точки $a_{i+1} = a_i + 1/2$, поэтому при большом количестве данных приближенно можно считать $a_N \approx N/2$. При расчете $b_{i+1}$ значение $(x-\mu_i) \approx \sigma$, поэтому приближенно для большого числа точек $b_N \approx N/2 \cdot \sigma^2/\sigma_0^2$. Для гамма-распределения среднее и дисперсия имеют вид $Mean(Gamma(\lambda; a_i, b_i)) = a_i / b_i$, $Var(Gamma(\lambda; a_i, b_i)) = a_i / b_i^2$. При большом количестве точек $a_N/b_N \approx \sigma_0^2/\sigma^2$, поэтому среднее $Mean(\lambda)$ стабилизируется вокруг $\sigma_0^2/\sigma^2$. Дисперсия $Var(\lambda) \approx \sigma_0^2/\sigma^2 b_i$ уменьшается с увеличением количества точек и ростом $b_i$.

Вместо совместного двухмерного распределения $P(\mu, \lambda | x)$ для анализа удобнее одномерные маржинальные распределения $P(\mu|x)$ и $P(\lambda|x)$. Маржинальное распределение $P(\lambda | x)$ по конструкции будет гамма-распределением:

$$
P(\lambda | x) = Gamma(\lambda; a_i, b_i).
$$

Можно показать, что маржинальное распределение $P(\mu | x)$ будет обобщенным $t$-распределением [[GenStudentT](https://en.wikipedia.org/wiki/Student's_t-distribution#Generalized_Student's_t-distribution), [SciPyT](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html)]. При интегрировании $P(\mu, \lambda)$ по $\lambda$:

$$
\begin{align}
P(\mu | x) 
= \int_0^\infty P(\mu, \lambda) d\lambda 
& = 
\int_0^\infty d\lambda 
\frac{(k_i \lambda)^{1/2}}{\sqrt{2 \pi \sigma_0^2}} e^{- \frac{k_i \lambda}{2 \sigma_0^2}(\mu - \mu_i)^2}
\frac{b_i^{a_i}}{\Gamma(a_i)} e^{-b_i \lambda} {\lambda}^{a_i-1}
\\
& \propto
\int_0^\infty d\lambda 
\lambda^{a_i + 1/2 - 1} e^{- \lambda \left( \frac{k_i}{2 \sigma_0^2}(\mu - \mu_i)^2 + b_i \right) } .
\end{align}
$$

Выражение под интегралом по форме совпадает с гамма-распределением. Т.к. $\int_0^\infty d\lambda Gamma(\lambda; \alpha, \beta) = 1$, то 
$\int_0^\infty d\lambda \lambda^{\alpha-1} e^{-\beta \lambda} = \Gamma(\alpha) {\beta}^{-\alpha}$.

После подстановки и выделения факторов, зависящих от $\mu$:
$$
\begin{align}
P(\mu | x) 
& \propto_{\mu} 
\left(b_i + \frac{k_i}{2 \sigma_0^2}(\mu - \mu_i)^2 \right)^{-(a_i + 1/2)}
\\
& \propto_{\mu} 
\left(1 + \frac{k_i}{2 \sigma_0^2 b_i}(\mu - \mu_i)^2 \right)^{-(a_i + 1/2)} .
\end{align}
$$

Обобщенное $t$-распределение имеет вид  
$$
t(x | \nu, \mu_t, \sigma_t^2) 
\propto_x 
\left(1+\frac{1}{\nu} \frac{ (x - \mu_t)^2 }{\sigma_t^2 } \right)^{-(\nu+1)/2} .
$$

Маржинальное распределение $P(\mu | x)$ задается $t$-распределением
$$
P(\mu | x) = t(\mu | \nu = 2a_i, \mu_t = \mu_i, \sigma_t^2 = \frac{\sigma_0^2}{k_i} \frac{b_i}{a_i} ) .
$$

Примеры гамма- и обобщенного $t$-распределений при различных параметрах приведены ниже. С ростом $a_i$ и $b_i$ при фиксированном отношении $a_i/b_i$ гамма-распределение локализуется вокруг этого значения, дисперсия $a_i / b_i^2$ сужается. В $t$-распределении с ростом $a_i$ увеличивается число степеней свободы $\nu$, что ведет к сужению распределения. Также по мере набора данных увеличивается $k_i$, что ведет к уменьшению $\sigma^2_t$ и снижению дисперсии.

In [ ]:
x = np.linspace(0, 10, 2000)
fig = go.Figure()
for a,b in [(3,1), (9, 3), (45, 15), (90, 30)]:
    fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=a, scale=1/b), mode='lines', name=f'a={a}, b={b}'))
fig.update_layout(title='Gamma Distribution',
                  xaxis_title='$\lambda$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


x = np.linspace(0, 10, 2000)
fig = go.Figure()
for df, loc, scale in [(4, 5, 1), (50, 5, 1), (50, 5, 1/2)]:
    fig.add_trace(go.Scatter(x=x, y=stats.t.pdf(x, df=df, loc=loc, scale=scale), 
                             mode='lines', 
                             name=f'nu={df}, mu={loc}, sigma_t={scale}'))
fig.update_layout(title='t-Distribution',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

# Приложение: оценка параметров нормального распределения по сэмплу

Пусть есть нормальное распределение $N(\mu, \sigma^2)$ с известными параметрами $\mu$ и $\sigma$. Есть сэмпл точек из него. Нужно по сэмплу оценить значения параметров и сравнить с реальными.

In [ ]:
mu_exact = 20
s_exact = 3
sample_size = 100

exact_dist = stats.norm(loc=mu_exact, scale=s_exact)
samp = exact_dist.rvs(size=sample_size)

x = np.linspace(0, 50, 2000)
fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mu_exact, scale=s_exact), mode='lines', name='Exact'))
fig.add_trace(go.Histogram(x=samp, histnorm='probability density', name='Sample'))
fig.update_layout(title='Dist',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

print(f'exact mu_0 = {mu_exact}, std={s_exact}')
print(f'sample avg = {np.mean(samp)}, std={np.std(samp)}')

В этом случае известно, что распределение данных нормальное. Поэтому для моделирования можно использовать нормальную функцию правдоподобия и сопряженное априорное распределение из раздела выше:

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)},
\\
P(data | model) = N(x; \mu, \frac{\sigma_{0}^2}{\lambda} ) ,  
\\
P(model) = N(\mu; \mu_i, \frac{\sigma_0^2}{k_i \lambda} ) Gamma(\lambda; a_i, b_i) . 
$$

Разбивать сэмпл на части и считать в них средние не нужно - распределение исходных данных уже нормальное. Способ задания начальных значений параметров $\mu_0, \sigma_0$ отличается. Можно либо использовать часть точек из сэмпла, либо просто выбрать для них близкие по порядку значения. Ниже используется второй вариант - $\mu_0$ и $\sigma_0$ задаются равными единице.

In [ ]:
mu_0 = 1
sigma_0 = 1

pars = []
pars.append(initial_parameters(mu=mu_0, sigma=sigma_0))

for x in samp:
    new_pars = update_conj_parameters(x, pars[-1])
    pars.append(new_pars)

print(f'Initial parameters: {pars[0]}')
print(f'Final parameters: {pars[-1]}')

Итоговые оценки распределений параметров:

In [ ]:
#mu
mu_dist = mu_marginal_distrib(pars[-1])
x = np.linspace(0, 50, 2000)
yplot = mu_dist.pdf(x)
ymax = max(yplot)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=mu_dist.pdf(x), mode='lines', name=f'Mu Estimate Distrib'))
fig.add_trace(go.Scatter(x=[samp.mean(), samp.mean()], y=[0, ymax],
                         mode='lines', line_color='black', line_dash='dash', 
                         name='Sample Mean'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, ymax], 
                         mode='lines', line_color='black',
                         name='Exact Mean'))
fig.update_layout(title='Mu Distribution',
                  xaxis_title='mu',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

# sigma
fig = go.Figure()
x = np.linspace(0.001, 10, 2000)
x_plot = x
y_plot = sigma_marginal_distrib_pdf(x, pars[-1])
y_max = max(y_plot)
fig.add_trace(go.Scatter(x=x, 
                         y=y_plot, 
                         mode='lines', line_dash='dash', name=f'Sigma Estimate Distrib'))
fig.add_trace(go.Scatter(x=[samp.std(), samp.std()], 
                         y=[0, y_max],
                         mode='lines', line_color='black', line_dash='dash', 
                         name='Sample Stdev'))
fig.add_trace(go.Scatter(x=[exact_dist.std(), exact_dist.std()], 
                         y=[0, y_max], 
                         mode='lines', line_color='black',
                         name='Exact Stdev'))
fig.update_layout(title='Sigma Distribution',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.show()


#posterior data
post_x = post_means_rvs(pars[-1], size=50000)
x = np.linspace(0, 50, 2000)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.add_vline(samp.mean(), line_dash='dash', name='Sample Mean')
fig.add_trace(go.Histogram(x=post_x, histnorm='probability density', name='Posterior', 
                           opacity=0.7, nbinsx=100))
fig.update_layout(title='Posterior Distribution',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.show()

Видно, что точные значения $\mu$ и $\sigma$ имеют ненулевую вероятность в построенных оценках распределений. Также апостериорное распределение близко к точному заданному.

Кроме итоговых значений на этом примере также полезно посмотреть динамику параметров. Можно ожидать, что среднее значение $\mu$ будет стабилизироваться вблизи среднего в сэмле, а дисперсия $\mu$ будет уменьшаться. Параметр $1/\lambda$ играет роль масштабирующего множителя между $\sigma_0^2$ и точным значением $\sigma^2$. Поэтому для распределения $\lambda$ также можно ожидать локализации вокруг отношения $\sigma_0^2/\sigma^2$. Можно проследить за этим по изменению среднего значения и стандартного отклонения распределения.

Аналитические выражения для среднего и дисперсии $\mu$, моды и дисперсии $\lambda$ приведены ниже [[GenStudentT](https://en.wikipedia.org/wiki/Student's_t-distribution#Generalized_Student's_t-distribution), [GammaDist](https://en.wikipedia.org/wiki/Gamma_distribution)]:

$$
P(\mu | data) = t(\mu | \nu = 2a_i, \mu_t = \mu_i, \sigma_t^2 = \frac{\sigma_0^2}{k_i} \frac{b_i}{a_i} ),
\\
Mean(\mu) = \mu_i, 
\quad
Var(\mu) = \frac{\sigma_0^2}{k_i} \frac{b_i}{a_i - 1} ,
$$

$$
P(\lambda | data) = Gamma(\lambda; a_i, b_i),
\\
Mean(\lambda) = \frac{a_i}{b_i},
\quad
Var(\lambda) = \frac{a_i}{b_i^2} .
$$

Динамика параметров в зависимости от количества учтенных точек:  

In [ ]:
pnts = [i for i in range(len(pars))]
mu_i = [p.mu for p in pars]
std_mu_i = [mu_marginal_distrib(p).std() for p in pars]
a_i = [p.a for p in pars]
b_i = [p.b for p in pars]
lmd_mean_i = [p.a / p.b for p in pars]
lmd_std_i = [lambda_marginal_distrib(p).std() for p in pars]

fig = make_subplots(rows=1, cols=2, subplot_titles=('Mu Mean',  'Mu Std'))
fig.add_trace(go.Scatter(x=pnts, y=mu_i), row=1, col=1)
fig.add_trace(go.Scatter(x=pnts, y=std_mu_i), row=1, col=2)
fig.update_layout(showlegend=False)
fig['layout']['xaxis'].update(title_text='Data Points')
fig['layout']['xaxis2'].update(title_text='Data Points')
fig.show()

fig = make_subplots(rows=1, cols=3, subplot_titles=('a_i, b_i', 'Lambda Mean',  'Lambda Std'))
fig.add_trace(go.Scatter(x=pnts, y=a_i, name='a'), row=1, col=1)
fig.add_trace(go.Scatter(x=pnts, y=b_i, name='b'), row=1, col=1)
fig.add_trace(go.Scatter(x=pnts, y=lmd_mean_i), row=1, col=2)
fig.add_trace(go.Scatter(x=pnts, y=lmd_std_i), row=1, col=3)
fig.update_layout(showlegend=False)
fig['layout']['xaxis'].update(title_text='Data Points')
fig['layout']['xaxis2'].update(title_text='Data Points')
fig['layout']['xaxis3'].update(title_text='Data Points')
fig.show()

Среднее $\mu$ стабилизируется, стандартное отклонение $\mu$ уменьшается. Кажется, что мода $\lambda$ также стабилизируется и стандартное отклонение $\lambda$ также уменьшается.

Вид распределений $\mu$ и $\sigma = \sigma_0 / \lambda^{1/2}$ для различного количества учтенных точек приведен ниже:

In [ ]:
i_points = [0, len(pars) // 20, len(pars) // 5, len(pars) // 2, len(pars) - 1]

x = np.linspace(0, 50, 2000)
y_plot_max = 0
fig = go.Figure()
for i in i_points:
    y_plot = mu_marginal_distrib(pars[i]).pdf(x)
    y_plot_max = max(max(y_plot), y_plot_max)
    fig.add_trace(go.Scatter(x=x, y=y_plot, mode='lines', name=f'i={i}'))
fig.add_trace(go.Scatter(x=[mu_exact, mu_exact], 
                         y=[0, y_plot_max], 
                         line_width=0.8, mode='lines', line_color='black', name='Exact Mean'))
fig.add_trace(go.Scatter(x=[np.mean(samp), np.mean(samp)],
                         y=[0, y_plot_max],
                         line_width=0.8, mode='lines', line_color='black', line_dash="dash", name='Sample Mean'))
fig.update_layout(title='Mu Distribution',
                  xaxis_title='$\mu$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


x = np.linspace(0.01, 10, 2000)
y_plot_max = 0
fig = go.Figure()
for i in i_points:
    y_plot = sigma_marginal_distrib_pdf(x, pars[i])
    y_plot_max = max(max(y_plot), y_plot_max)
    fig.add_trace(go.Scatter(x=x_plot, y=y_plot, mode='lines', name=f'i={i}'))
fig.add_trace(go.Scatter(x=[s_exact, s_exact], 
                         y=[0, y_plot_max], 
                         line_width=0.8, mode='lines', line_color='black', name='Exact Stdev'))
fig.add_trace(go.Scatter(x=[np.std(samp), np.std(samp)],
                         y=[0, y_plot_max],
                         line_width=0.8, mode='lines', line_color='black', line_dash="dash", name='Sample Stdev'))
fig.update_layout(title='Sigma Distribution',
                  xaxis_title='$\sigma$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 10])
fig.show()

Видно как по мере учета данных распределения $\mu$ и $\sigma$ смещаются к соответствующим значениям в сэмпле и сужаются.

**todo: Приложение: выбросы**  
*Для функции правдоподобия попробовать использовать обратное гамма-распределение вместо нормального (см. [вопрос на SO](https://stats.stackexchange.com/questions/593923/what-likelihood-to-use-to-model-sample-means-from-a-pareto-like-distribution)). Распределение параметров оценить численно.*


# Заключение

Показано, как в рамках байесовского подхода можно проводить сравнение средних различных метрик в А/Б-тестах. Метод полагается на центральную предельную теорему. Данные разбиваются на части, в каждой части считается среднее, распределение частичных средних моделируется нормальным распределением. Оптимальная группа выбирается при сравнении распределений параметров. Эксперимент останавливается при достижении заданной вероятности того, что значение параметра в одной группе больше другой. Приводится способ оценки длительности эксперимента до достижения этого условия.

Некоторые не обсуждавшиеся вопросы обозначены ниже.

Стоит промоделировать количество правильно угаданных "лучших" вариантов при использовании предложенного метода и требующееся для этого количество точек. 

Центральная предельная теорема применима к распределениям с конечными средним и дисперсией. Формально никто из пользователей не проведет на сервисе бесконечно много времени и не потратит бесконечно много денег. Тем не менее в экспериментальных данных могут быть выбросы. В таком случае для приближения частичных средних нормальным распределением нужно считать выборочные средние по "большому" количество точек. Оценку этого числа можно получить с помощью неравенства Берри-Эссеена. Также после выбора определенного количества точек для расчета выборочных средних требуется валидация нормального распределения для моделирования их распределения.  

Средние величины не всегда являются адекватными характеристиками распределения. Иногда может быть нужна более детальная информация. В этом случае нужно выбирать модель под каждое конкретное распределение.  

Критерием остановки эксперимента выбиралось достижение заданного уровня уверенности в том, что значение параметра в одной группе больше другой. Не обсуждалось проведение эксперимента до падения ценности новых данных ниже цены их получения.

# Благодарности

# Ссылки

[SGBS] B. Lambert, A Student’s Guide to Bayesian Statistics ( [Textbook](https://www.amazon.co.uk/Students-Guide-Bayesian-Statistics/dp/1473916364), [Student Resources](https://study.sagepub.com/lambert) )     
[SR] R. McElreath, Statistical Rethinking: A Bayesian Course with Examples in R and STAN ([Textbook](https://www.amazon.co.uk/Statistical-Rethinking-Bayesian-Examples-Chapman/dp/036713991X/ref=sr_1_1), [Video Lectures](https://www.youtube.com/playlist?list=PLDcUM9US4XdMROZ57-OIRtIK0aOynbgZN), [Course Materials](https://github.com/rmcelreath/stat_rethinking_2022) )  
[BTYD] - [Buy Till You Die](https://en.wikipedia.org/wiki/Buy_Till_you_Die), *Wikipedia.*  
[BayesAB1] - [1-General Approach](https://nbviewer.org/github/noooway/Bayesian_ab_testing/blob/main/1-%D0%9E%D0%B1%D1%89%D0%B8%D0%B5%20%D0%B8%D0%B4%D0%B5%D0%B8.ipynb) in Bayesian A/B Testing, *GitHub.*  
[CLT] - [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem), *Wikipedia.*  
[NormDist] - [Normal Distribution](https://en.wikipedia.org/wiki/Normal_distribution), *Wikipedia.*   
[SciPyNorm] - [scipy.stats.norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html?highlight=norm), *SciPy Reference.*   
[RandVarsConv] - [Convergence in distribution](https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_distribution) in *Convergence of random variables*, *Wikipedia.*  
[BerryEsseenTheorem] - [Berry-Esseen Theorem](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem), *Wikipedia.*  
[ParetoDist] - [Pareto Distribution](https://en.wikipedia.org/wiki/Pareto_distribution), *Wikipedia.*  
[SciPyPareto] - [scipy.stats.pareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html), *SciPy Reference.*   
[LomaxDist] - [Lomax Distribution](https://en.wikipedia.org/wiki/Lomax_distribution), *Wikipedia.*   
[SciPyLomax] - [scipy.stats.lomax](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lomax.html), *SciPy Reference.*   
[GenCLT] - [A generalized central limit theorem](https://en.wikipedia.org/wiki/Stable_distribution#A_generalized_central_limit_theorem) in *Stable Distribution*, *Wikipedia.*  
[GammaDist] - [Gamma Distribution](https://en.wikipedia.org/wiki/Gamma_distribution), *Wikipedia.*  
[SciPyStatsGamma] - [scipy.stats.gamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html), *SciPy Reference.*   
[GenStudentT] - [Generalized Student's t-distribution](https://en.wikipedia.org/wiki/Student's_t-distribution#Generalized_Student's_t-distribution) in *Student's t-distribution*, *Wikipedia.*   
[SciPyT] - [scipy.stats.t](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html), *SciPy Reference.*   
[ProbChangeVars] - [Probability Density Function](https://en.wikipedia.org/wiki/Probability_density_function#Function_of_random_variables_and_change_of_variables_in_the_probability_density_function), *Wikipedia.*  
[ConjNormal] - K.P. Murphy, *[Conjugate Bayesian analysis of the Gaussian distribution](https://www.cs.ubc.ca/~murphyk/Papers/bayesGauss.pdf)*, 2007.  